# MIDS 251 Final Project
## Analyzing Reddit Data
### Creating a prediction model for subreddit using pyspark.ml.DecisionTreeClassifier

In [1]:
import os
import sys

In [2]:
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.8.2.1-src.zip"))
os.chdir(spark_home)

In [3]:
execfile(os.path.join(spark_home, "python/pyspark/shell.py"))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.2
      /_/

Using Python version 2.7.5 (default, Jun 24 2015 00:41:19)
SparkContext available as sc, SQLContext available as sqlContext.


In [4]:
import os
os.getcwd()


'/usr/local/spark-1.5.3-SNAPSHOT-bin-spark-1.5.2-hadoop-2.6.2'

In [13]:
spark_home = os.environ.get('SPARK_HOME', None)
text_file = sc.textFile("file:///" + spark_home + "/README.md")

In [15]:
word_counts = text_file \
   .flatMap(lambda line: line.split()) \
   .map(lambda word: (word, 1)) \
   .reduceByKey(lambda a, b: a + b)

In [20]:
for wc in word_counts.collect():
    print(wc)

(u'when', 1)
(u'R,', 1)
(u'including', 3)
(u'computation', 1)
(u'using:', 1)
(u'guidance', 3)
(u'Scala,', 1)
(u'environment', 1)
(u'only', 1)
(u'rich', 1)
(u'Apache', 1)
(u'sc.parallelize(range(1000)).count()', 1)
(u'Building', 1)
(u'guide,', 1)
(u'return', 2)
(u'Please', 3)
(u'Try', 1)
(u'not', 1)
(u'Spark', 14)
(u'scala>', 1)
(u'Note', 1)
(u'cluster.', 1)
(u'./bin/pyspark', 1)
(u'have', 1)
(u'params', 1)
(u'through', 1)
(u'GraphX', 1)
(u'[run', 1)
(u'abbreviated', 1)
(u'[project', 2)
(u'##', 8)
(u'library', 1)
(u'see', 1)
(u'"local"', 1)
(u'[Apache', 1)
(u'will', 1)
(u'#', 1)
(u'processing,', 1)
(u'for', 12)
(u'[building', 1)
(u'provides', 1)
(u'print', 1)
(u'supports', 2)
(u'built,', 1)
(u'[params]`.', 1)
(u'available', 1)
(u'run', 7)
(u'tests](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools).', 1)
(u'This', 2)
(u'Hadoop,', 2)
(u'Tests', 1)
(u'example:', 1)
(u'-DskipTests', 1)
(u'Maven](http://maven.apache.org/).', 1)
(u'programming', 1)
(u'running', 1)
(u'a

In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils

# Load and parse the data file, converting it to a DataFrame.
data = MLUtils.loadLibSVMFile(sc, "file:///usr/local/spark/data/mllib/sample_libsvm_data.txt").toDF()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")
accuracy = evaluator.evaluate(predictions)
print "Test Error = %g" % (1.0 - accuracy)

treeModel = model.stages[2]
print treeModel # summary only

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[129,130,131...|
|       0.0|         0.0|(692,[158,159,160...|
|       1.0|         1.0|(692,[154,155,156...|
|       0.0|         0.0|(692,[129,130,131...|
|       0.0|         0.0|(692,[97,98,99,12...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0
DecisionTreeClassificationModel of depth 2 with 5 nodes
